# Create Protein Design Agent with AWS HealthOmics Workflow Integration

This notebook demonstrates how to create a Bedrock agent that can trigger AWS HealthOmics workflows for protein design optimization.

# Pre-requisites

1. Go through the notebook environment setup in the agents_catalog/0-Notebook-environment/ folder

2. Deploy cf_template.yaml to your AWS account to instantiate a ECR repository with a custom Docker image, a AWS HealthOmics (AHO) private workflow, and a lambda function that invokes the AHO workflow


### Steps for deploying the CloudFormation stack:
1. Upload workflow definition files to S3
2. Package and upload container code to S3
3. Deploy the CloudFormation stack

In [ ]:
import os
import shutil
import boto3


# Create zip file of container code
def create_container_zip():
    try:
        shutil.make_archive('code', 'zip', 'container')
        print("Successfully created code.zip from container directory")
    except Exception as e:
        print(f"Error creating zip file: {e}")

# Upload workflow files and container code to S3
def upload_to_s3(bucket_name):
    s3 = boto3.client('s3')
    
    # Upload workflow files
    workflow_files = ['main.nf', 'nextflow.config', 'config.yaml', 'parameter-template.json']
    for file in workflow_files:
        try:
            s3.upload_file(
                f'aho_workflow/{file}', 
                bucket_name, 
                f'workflow/{file}'
            )
            print(f"Uploaded {file} to s3://{bucket_name}/workflow/")
        except Exception as e:
            print(f"Error uploading {file}: {e}")
    
    # Upload container code zip
    try:
        s3.upload_file(
            'code.zip',
            bucket_name,
            'code.zip'
        )
        print(f"Uploaded code.zip to s3://{bucket_name}/")
    except Exception as e:
        print(f"Error uploading code.zip: {e}")

# Create zip and upload files
s3_bucket_name = "hcls-bedrock-agents-byot-aho"
create_container_zip()
upload_to_s3(s3_bucket_name)


### AWS CLI commands to deploy the CloudFormation stack:
````
# Deploy the CloudFormation stack
aws cloudformation create-stack \\
    --stack-name hcls-bedrock-agents-byot-aho-stack \\
    --template-body file://cf_template.yaml \\
    --parameters file://cf_parameters.json \\
    --capabilities CAPABILITY_IAM CAPABILITY_AUTO_EXPAND CAPABILITY_NAMED_IAM \
    --region us-east-1

# Monitor stack creation
aws cloudformation describe-stacks \\
    --stack-name hcls-bedrock-agents-byot-aho-stack \\
    --query 'Stacks[0].StackStatus'

# Get stack outputs once complete
aws cloudformation describe-stacks \\
    --stack-name hcls-bedrock-agents-byot-aho-stack \\
    --query 'Stacks[0].Outputs'
```

#### Load in environment variables to notebook

In [1]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

%run $IMPORTS_PATH

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Successfully imported necessary libraries into notebook


## Configure AWS clients and parameters

In [2]:
import boto3
import json
import time
import uuid
from botocore.exceptions import ClientError

# Explicitly specify the region
REGION = 'us-east-1'  # Replace with your stack's region
# Configure AWS clients
session = boto3.Session()
account_id = boto3.client('sts').get_caller_identity()['Account']

bedrock = boto3.client('bedrock', REGION)
cfn = boto3.client('cloudformation', REGION)

## Get CloudFormation Outputs

In [3]:
import boto3
from botocore.exceptions import ClientError
import json

STACK_NAME = 'hcls-bedrock-agents-byot-aho-stack'

# Initialize the CloudFormation client with the specific region
cloudformation = boto3.client('cloudformation', region_name=REGION)

def get_cloudformation_outputs(stack_name):
    try:
        response = cloudformation.describe_stacks(StackName=stack_name)
        outputs = {}
        for output in response['Stacks'][0]['Outputs']:
            outputs[output['OutputKey']] = output['OutputValue']
        return outputs
    except ClientError as e:
        print(f"Error getting CloudFormation outputs: {e}")
        raise

# Get the outputs from CloudFormation
cf_outputs = get_cloudformation_outputs(STACK_NAME)
print("CloudFormation Outputs:")
print(json.dumps(cf_outputs, indent=2))

CloudFormation Outputs:
{
  "TriggerFunctionArn": "arn:aws:lambda:us-east-1:851725420776:function:hcls-bedrock-agents-byot-a-WorkflowTriggerFunction-VGHcrbVd03j4",
  "WorkflowExecutionRoleArn": "arn:aws:iam::851725420776:role/hcls-bedrock-agents-byot-aho--WorkflowExecutionRole-VrUcU3Yr1Wzq",
  "ECRRepositoryUri": "851725420776.dkr.ecr.us-east-1.amazonaws.com/protein-design-evoprotgrad"
}


In [4]:
lambda_function_arn = cf_outputs["TriggerFunctionArn"]
lambda_function_name = "hcls-bedrock-agents-byot-a-WorkflowTriggerFunction"

## Create Bedrock Agent

In [5]:
# Define agent configuration
agent_name = 'ProteinDesignAgent'
agent_description = "Agent for protein design using HealthOmics workflow"
agent_instruction = """You are an expert in protein design and optimization using AWS HealthOmics workflows. 
Your primary task is to help users run protein design optimization workflows and provide relevant insights.

When providing your response:
a. Start with a brief summary of your understanding of the user's query.
b. Explain the steps you're taking to address the query. Ask for clarifications from the user if required.
c. Present the results of the workflow execution."""


In [6]:
agent_foundation_model

['anthropic.claude-3-5-sonnet-20241022-v2:0']

## Create Agent Instance

In [8]:
# Instantiate agent with the desired configuration
agents = AgentsForAmazonBedrock()

protein_design_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

# Extract useful agent information
protein_design_agent_id = protein_design_agent[0]
protein_design_agent_arn = f"arn:aws:bedrock:{REGION}:{account_id}:agent/{protein_design_agent_id}"

print(f"Agent created with ID: {protein_design_agent_id}")
print(f"Agent ARN: {protein_design_agent_arn}")

Agent created with ID: 1LHK282YZV
Agent ARN: arn:aws:bedrock:us-east-1:851725420776:agent/1LHK282YZV


## Define Action Group Functions

In [9]:
function_defs = [
    {
        "name": "trigger_aho_workflow",
        "description": "Trigger the AWS HealthOmics workflow for protein design optimization",
        "parameters": {
            "workflowId": {
                "description": "The ID of the HealthOmics workflow to run",
                "required": True,
                "type": "string"
            },
            "runName": {
                "description": "Name for the workflow run",
                "required": True,
                "type": "string"
            },
            "container_image": {
                "description": "ECR image URI for the protein design container",
                "required": True,
                "type": "string"
            },
            "seed_sequence": {
                "description": "The input protein sequence to optimize",
                "required": True,
                "type": "string"
            },
            "outputUri": {
                "description": "S3 URI where the workflow outputs will be stored",
                "required": True,
                "type": "string"
            },
            "roleArn": {
                "description": "ARN of the IAM role for workflow execution",
                "required": True,
                "type": "string"
            }
        },
        "requireConfirmation": "DISABLED"
    }
]


## Add Action Group with Lambda Function

In [10]:
# Add action group with Lambda function
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=lambda_function_name,
    source_code_file=lambda_function_arn,
    agent_action_group_name="ProteinDesignActions",
    agent_action_group_description="Actions for protein design using AWS HealthOmics workflows",
    agent_functions=function_defs,
    verbose=True
)

Creating action group: ProteinDesignActions...
Lambda ARN: arn:aws:lambda:us-east-1:851725420776:function:hcls-bedrock-agents-byot-a-WorkflowTriggerFunction-VGHcrbVd03j4
Agent functions: [{'name': 'trigger_aho_workflow', 'description': 'Trigger the AWS HealthOmics workflow for protein design optimization', 'parameters': {'workflowId': {'description': 'The ID of the HealthOmics workflow to run', 'required': True, 'type': 'string'}, 'runName': {'description': 'Name for the workflow run', 'required': True, 'type': 'string'}, 'container_image': {'description': 'ECR image URI for the protein design container', 'required': True, 'type': 'string'}, 'seed_sequence': {'description': 'The input protein sequence to optimize', 'required': True, 'type': 'string'}, 'outputUri': {'description': 'S3 URI where the workflow outputs will be stored', 'required': True, 'type': 'string'}, 'roleArn': {'description': 'ARN of the IAM role for workflow execution', 'required': True, 'type': 'string'}}, 'requireC

## Add Lambda Resource-Based Policy

In [11]:
lambda_client = boto3.client('lambda', REGION)

try:
    # Add the new statement to the existing policy
    response = lambda_client.add_permission(
        FunctionName=lambda_function_arn,
        StatementId="AllowBedrockAgentAccess",
        Action="lambda:InvokeFunction",
        Principal="bedrock.amazonaws.com",
        SourceArn=protein_design_agent_arn
    )
    
    print("Resource policy added successfully.")
    print("Response:", response)
except lambda_client.exceptions.ResourceConflictException:
    print("Permission already exists")
except Exception as e:
    print(f"Error adding permission: {e}")

Permission already exists


## Create Agent Alias

In [12]:
# Create agent alias
protein_design_agent_alias_id, protein_design_agent_alias_arn = agents.create_agent_alias(
    protein_design_agent[0], 'v1'
)

# Store the alias ARN for future use
%store protein_design_agent_alias_arn

print(f"Agent alias created with ID: {protein_design_agent_alias_id}")
print(f"Agent alias ARN: {protein_design_agent_alias_arn}")

Stored 'protein_design_agent_alias_arn' (str)
Agent alias created with ID: QOR1QGH1NR
Agent alias ARN: arn:aws:bedrock:us-west-2:851725420776:agent-alias/1LHK282YZV/QOR1QGH1NR


## Test the Agent

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", REGION)
session_id = str(uuid.uuid1())

test_query = "Can you help me optimize a protein sequence?"

response = bedrock_agent_runtime_client.invoke_agent(
    inputText=test_query,
    agentId=protein_design_agent_id,
    agentAliasId=protein_design_agent_alias_id,
    sessionId=session_id,
    enableTrace=True
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

print("Agent Answer: {}".format(answer))
print("====================")

Request sent to Agent:
{'ResponseMetadata': {'RequestId': 'e490c8df-211c-4a53-96b4-4f8aa5ee8030', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 03 Apr 2025 23:37:44 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'e490c8df-211c-4a53-96b4-4f8aa5ee8030', 'x-amz-bedrock-agent-session-id': 'a4685cc8-10e4-11f0-928e-b6013b69c924', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': 'a4685cc8-10e4-11f0-928e-b6013b69c924', 'completion': <botocore.eventstream.EventStream object at 0x7feaec9400d0>}
Agent processing query now


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 answer = ""                                                                                 │
│   21                                                                                             │
│   22 # Iterate through the event stream                                                          │
│ ❱ 23 for event in response['completion']:                                                        │
│   24 │   # Check if the event is a 'chunk' event                                                 │
│   25 │   if 'chunk' in event:                                                                    │
│   26 │   │   chunk_obj = event['chunk']                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/eventstream.py:592 in __iter__                  │
│                                                                                                  │
│   589 │                                                                                          │
│   590 │   def __iter__(self):                                                                    │
│   591 │   │   for event in self._event_generator:                                                │
│ ❱ 592 │   │   │   parsed_event = self._parse_event(event)                                        │
│   593 │   │   │   if parsed_event:                                                               │
│   594 │   │   │   │   yield parsed_event                                                         │
│   595                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/botocore/eventstream.py:608 in _parse_event              │
│                                                                                                  │
│   605 │   │   if response_dict['status_code'] == 200:                                            │
│   606 │   │   │   return parsed_response                                                         │
│   607 │   │   else:                                                                              │
│ ❱ 608 │   │   │   raise EventStreamError(parsed_response, self._operation_name)                  │
│   609 │                                                                                          │
│   610 │   def get_initial_response(self):                                                        │
│   611 │   │   try:                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
EventStreamError: An error occurred (resourceNotFoundException) when calling the InvokeAgent operation: Failed to 
retrieve resource because it doesn't exist. Retry the request with a different resource identifier.

> /opt/conda/lib/python3.11/site-packages/botocore/eventstream.py(608)_parse_event()
    606             return parsed_response
    607         else:
--> 608             raise EventStreamError(parsed_response, self._operation_name)
    609 
    610     def get_initial_response(self):



ipdb>  parsed_response


{'Error': {'Code': 'resourceNotFoundException', 'Message': "Failed to retrieve resource because it doesn't exist. Retry the request with a different resource identifier."}}


...

ipdb>  event


ipdb>  d


> /opt/conda/lib/python3.11/site-packages/botocore/eventstream.py(608)_parse_event()
    606             return parsed_response
    607         else:
--> 608             raise EventStreamError(parsed_response, self._operation_name)
    609 
    610     def get_initial_response(self):



ipdb>  parsed_response


{'Error': {'Code': 'resourceNotFoundException', 'Message': "Failed to retrieve resource because it doesn't exist. Retry the request with a different resource identifier."}}


In [ ]:
{
  "workflowId": "8811413",
  "runName": "test-protein-design-run",
  "parameters": {
    "container_image": "851725420776.dkr.ecr.us-east-1.amazonaws.com/protein-design-evoprotgrad:latest",
    "seed_sequence": "MKFLILLFNILCLFPVLAADNHGVGPQGASGVDPITFDINSNQTGPAFLTAVEMAGVKYLQVQHGSNVNIHRLVEGNVVIWENASTPLYTGAIVTNNDGPYMAYVEVLGDPNLQFFIKSGDAWVTLSEHEYLAKLQEIRQAVHIESVFSLNMAFQLENNKYEVETHAKNGANMVTFIPRNGHICKMVYHKNVRIYKATGPTETQNLQVLFKTAGVIPENKDWWHIFKASRVMKGLEDVDILQCLYVNLYTMITPMLNPFIYSLRNRDTLLASDNAGFGAERDGSCPEAPMCYTIDVNMNMAVRGLFVRPQIPLTGYHGQEFFFKDQRGIHHHHHH"
  },
  "outputUri": "s3://hcls-bedrock-agents-byot-aho/output/",
  "roleArn": "arn:aws:iam::851725420776:role/hcls-bedrock-agents-byot-aho--WorkflowExecutionRole-VrUcU3Yr1Wzq"
}